<a href="https://colab.research.google.com/github/manipta/Spam_Ham_Filter/blob/main/ass6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Importing Dataset

In [33]:
import pandas as pd
sms_data=pd.read_csv('spam.csv',encoding = 'ISO-8859-1', engine = "python")
sms_data.groupby('v1').count()

,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
v1,,,,
ham,4825,45,10,6
spam,747,5,2,0


#2.Data Preprocessing

In [34]:
sms_data_clean=pd.read_csv('spam.csv',encoding = 'ISO-8859-1', engine = "python")
sms_data_clean['v2']= sms_data_clean['v2'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()
sms_data_clean['v2']= sms_data_clean['v2'].str.lower()
sms_data_clean['v2']= sms_data_clean['v2'].str.split()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [35]:
sms_data_clean

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"[go, until, jurong, point, crazy, available, o...",NaN,NaN,NaN
1,ham,"[ok, lar, joking, wif, u, oni]",NaN,NaN,NaN
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...",NaN,NaN,NaN
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",NaN,NaN,NaN
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,"[this, is, the, 2nd, time, we, have, tried, 2,...",NaN,NaN,NaN
5568,ham,"[will, ì_, b, going, to, esplanade, fr, home]",NaN,NaN,NaN
5569,ham,"[pity, was, in, mood, for, that, so, any, othe...",NaN,NaN,NaN
5570,ham,"[the, guy, did, some, bitching, but, i, acted,...",NaN,NaN,NaN


In [36]:
 
train_data = sms_data_clean.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = sms_data_clean.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

sms_data_clean['v1'].value_counts() / sms_data.shape[0] * 100
'''
ham     86.593683
spam    13.406317
Name: Label, dtype: float64
'''
train_data['v1'].value_counts() / train_data.shape[0] * 100
'''
ham     86.54105
spam    13.45895
Name: Label, dtype: float64
'''
test_data['v1'].value_counts() / test_data.shape[0] * 100
'''
ham     86.983842
spam    13.016158
Name: Label, dtype: float64
'''

'\nham     86.983842\nspam    13.016158\nName: Label, dtype: float64\n'

In [37]:
vocabulary = list(set(train_data['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

#3. Final Function

In [39]:

def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1
      
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['v1'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1


In [40]:
Pspam = train_data['v1'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['v1'].value_counts()['ham'] / train_data.shape[0]
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'